# Preâmbulo

Imports, funções, downloads e instalação do Pytorch.

In [ ]:
# Basic imports.
import os
import time
import numpy as np

import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.utils import data
from torch.backends import cudnn

from torchvision import transforms
from torchvision import datasets
from torchvision import models

from skimage import io
from skimage import transform

from sklearn import metrics

from matplotlib import pyplot as plt

%matplotlib inline

cudnn.benchmark = True

In [ ]:
# Setting predefined arguments.
args = {
    'epoch_num': 50,      # Number of epochs.
    'n_classes': 3, #19,      # Number of classes in segmentation task.
    'pretrained': True,   # Boolean indicating pretraining of backbone.
    'skip': True,         # Boolean indicating presence of Skip Connections.
    'lr': 1e-4,           # Learning rate.
    'weight_decay': 5e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 3,     # Number of workers on data loader.
    'batch_size': 8,      # Mini-batch size.
    'w_size': 224,        # Width size for image resizing.
    'h_size': 224,        # Height size for image resizing.
    'show_freq': 5,      # Show predictions in images each show_freq epochs.
}

if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

Pré-definindo uma função para iniciação aleatória de pesos.

In [ ]:
# Random initialization for weights and biases.
def initialize_weights(*models):
    for model in models:
        for module in model.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight)
                if module.bias is not None:
                    module.bias.data.zero_()
            elif isinstance(module, nn.BatchNorm2d):
                module.weight.data.fill_(1)
                module.bias.data.zero_()

## Métricas usadas para Segmentação Semântica

As métricas mais utilizadas na área de segmentação semântica (e em algumas aplicações em terefas de detecção também) são variações da Intersection over Union ($IoU$), também chamada de **Jaccard**.

![IoU](https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Intersection_over_Union_-_poor%2C_good_and_excellent_score.png/300px-Intersection_over_Union_-_poor%2C_good_and_excellent_score.png)

Em tarefas de segmentação/detecção binária, se usa a $IoU$ simples, dada pela fórmula:

$IoU = \frac{\|A\ \cap\ B\|}{\|A\ \cup\ B\|} = \frac{TP}{TP + FP + FN}$.

No caso de problemas multiclasse, usa-se a média do $IoU$ para todas as classes, chamada de mean Intersection over Union ($mIoU$).

A métrica F1 (conhecida também como **Dice**) também é utilizada nesses contextos e é dada pela fórmula:

$F1 = \frac{2TP}{2TP + FP + FN}$.

In [ ]:
# Computing mean Intersection over Union (mIoU).
def evaluate(prds, labs):
    
    int_sum = np.zeros(args['n_classes'], dtype=np.float32)
    uni_sum = np.zeros(args['n_classes'], dtype=np.float32)
    
    for prd, lab in zip(prds, labs):
        
        for c in range(args['n_classes']):

            union = np.sum(lab.ravel() == c)
            if union > 0:
                uni_sum[c] += union
                
                intersection = np.sum(np.logical_and(lab.ravel() == c, prd.ravel() == c))
                int_sum[c] += intersection
    
    return int_sum, uni_sum

# O dataset Pascal VOC 2012

O dataset da competição [Pascal VOC 2012](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/) contém rótulos para detecção e segmentação de imagens, como pode ser visto nas imagens abaixo. Ele e o dataset [COCO](http://cocodataset.org/) são os principais benchmarks da área de Visão Computacional atualmente em tarefas de segmentação semântica e detecção em imagens.

![](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/segexamples/images/21.jpg)![](http://host.robots.ox.ac.uk/pascal/VOC/voc2012/segexamples/images/21_class.png)

O Pytorch possui um dataloader específico pro Pascal VOC 2012.

In [ ]:
# Root directory for VOC.
root = './'

# Classes left in Pascal VOC 2012.
voc_classes = [0, 89, 147]

# All classes in Pascal VOC 2012.
# voc_classes = [0, 14, 19, 33, 37, 38, 52, 57, 72, 75, 89, 94, 108, 112, 113, 128, 132, 147, 150, 220]


# Custom Image Transform.
class VOCImgTransform(object):
    
    def __call__(self, img):
#         img = transforms.functional.five_crop(img, (args['h_size'], args['w_size']))
        img = transforms.functional.center_crop(img, (args['h_size'], args['w_size']))
        img = transforms.functional.to_tensor(img)
        img = transforms.functional.normalize(img,
                                              mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
        
        return img

# Custom Target Transform.
class VOCTarTransform(object):
    
    def to_sequential_labels(self, tar):
        
        for i, c in enumerate(voc_classes):
            tar[tar == c] = i
        
        tar[tar > 2] = 0
        return tar
    
    def __call__(self, tar):
        tar = tar.convert(mode='I')
        tar = transforms.functional.center_crop(tar, (args['h_size'], args['w_size']))
        tar = transforms.functional.to_tensor(tar).squeeze().type(torch.int64)
        
        tar = self.to_sequential_labels(tar)
        
        return tar


# Setting datasets.
download_voc = not os.path.isdir('./VOCdevkit/')
train_set = datasets.VOCSegmentation(root,
                                     image_set='train',
                                     download=download_voc,
                                     transform=VOCImgTransform(),
                                     target_transform=VOCTarTransform())
val_set = datasets.VOCSegmentation(root,
                                   image_set='val',
                                   download=False,
                                   transform=VOCImgTransform(),
                                   target_transform=VOCTarTransform())

Exibindo algumas imagens do Pascal VOC 2012 e seus respectivos rótulos a nível de pixel.

In [ ]:
# Filtering VOC dataset to iterate only over samples from predefined classes.
train_indices = []
val_indices = []

valid_classes = [0, 1, 2]
    
for i, batch_data in enumerate(train_set):
    
    imgs, labs = batch_data
    
    curr_labels = set(list(labs.numpy().ravel()))
    
    if len([c for c in valid_classes if c in curr_labels]) >= 2: 
    
        train_indices.append(i)
        
        if i < 50:
            
            fig, ax = plt.subplots(1, 2, figsize=(16, 8))
    
            ax[0].imshow(imgs.numpy().transpose(1, 2, 0))
            ax[0].set_yticks([])
            ax[0].set_xticks([])
            ax[0].set_title('Image')

            ax[1].imshow(labs.numpy().squeeze() * 127, cmap=plt.get_cmap('gray'),
                         vmin=0, vmax=255)
            ax[1].set_yticks([])
            ax[1].set_xticks([])
            ax[1].set_title('Mask')

            plt.show()
            
for i, batch_data in enumerate(val_set):
    
    imgs, labs = batch_data
    
    curr_labels = set(list(labs.numpy().ravel()))
    
    if len([c for c in valid_classes if c in curr_labels]) >= 2:
    
        val_indices.append(i)

print(len(train_set), len(train_indices))
print(len(val_set), len(val_indices))

Definindo dataloader customizado baseado no subsample do dataset padrão Pascal VOC 2012.

In [ ]:
# Sampler for limiting classes to humans and automobiles.
sampler_train = data.SubsetRandomSampler(train_indices)
sampler_val = data.SubsetRandomSampler(val_indices)

# Setting dataloaders.
train_loader = DataLoader(train_set,
                          batch_size=args['batch_size'],
                          num_workers=args['num_workers'],
                          sampler=sampler_train)
val_loader = DataLoader(val_set,
                        batch_size=1,
                        num_workers=args['num_workers'],
                        sampler=sampler_val)

# Fully Convolutional Networks

[Fully Convolutional Networks (FCNs)](https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf) foram propostas em 2015 com o intuito de reaproveitar as CNNs pré-treinadas que já existiam para classificação de imagens como a [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) e a [VGG](https://arxiv.org/abs/1409.1556) em tarefas de **segmentação**. Como temos visto nas últimas aulas, CNNs tradicionais possuem vários blocos convolucionais e de pooling no começo da arquitetura, seguidos de blocos Fully Connected (FC) no fim da rede, os quais realizam a tarefa de inferência. Uma tarefa de segmentação é essencialmente uma classificação de pixels, também conhecida como uma tarefa de **rotulação densa**, contrária à **rotulação esparsa** de uma task de classificação de imagens, em que há um único rótulo por imagem.

![AlexNet](https://miro.medium.com/max/686/0*xPOQ3btZ9rQO23LK.png)

![VGG](https://miro.medium.com/max/700/0*V1muWIDnPVwZUuEv.png)

Também já fomos introduzidos ao conceito de Transfer Learning, que consiste de reaproveitar o conhecimento de um domínio e transferí-lo para outro domínio relativamente semelhante. Transfer Learning pode ser aplicado para transferir o conhecimento obtido num dataset como o ImageNet (para classificação de imagens) para uma tarefa de segmentação num dataset como o Pascal VOC 2012. Como o nome indica, **Fully Convolutional Networks** possuem apenas camadas convolucionais, substituindo as camadas FC por mais blocos convolucionais.

![CNN para FCN](https://www.dropbox.com/s/qsovwd3vuqxkhka/fcns.png?dl=1)

Percebe-se que à medida que as imagens vão passando pelos blocos convolucionais, vão perdendo resolução espacial e sendo representados por cada vez mais canais numa CNN tradicional. No caso das FCNs tradicionais, utiliza-se uma interpolação bilinear simples nos eixos $(H, W)$ nos feature maps que saem da última camada convolucional da CNN tradicional e entram nas novas camadas convolucionais que substituíram as FCs. Por exemplo, no caso da **AlexNet**, os feature maps que saem da última camada convolucional possuem dimensões $(B, C, H, W) = (B, 256, 7, 7)$. Após a interpolação bilinear, esse feature map passa a ter $(B, 256, 224, 224)$, com as últimas dimensões correspondendo à dimensão de entrada $(H_{in}, W_{in}) = (224, 224)$ das imagens do ImageNet.

A última camada de uma FCN (ou de basicamente qualquer outra rede para segmentação semântica) deve ser uma convolução com o número de canais igual ao número de classes. Isso faz com que cada canal de saída sirva como preditor de uma classe numa FCN. A saída da rede deve ter, portanto, as dimensões $(B, \#classes, H_{in}, W_{in})$, de forma que, se for passado um softmax na dimensão 1 (referente a $\#classes$), obtenha-se uma distribuição de probabilidades das classes para cada pixel. Dessa forma, conseguimos transformar uma rede que trata um problema de rotulação esparsa (como uma CNN para classificação) em uma rede que realiza rotulação densa.

FCNs tradicionais partem do pressuposto de que toda a **informação espacial** da imagem original está codificada nos canais que saem das camadas convolucionais da CNN que serve de backbone para a arquitetura. Isso pode não ser verdade principalmente para arquiteturas mais profundas, nas quais há uma discrepância muito grande as dimensões espaciais das imagens de entrada e as dimensões do output da última camada convolucional (i.e. de $(224,224)$ para $(7,7)$). Portanto, no próprio paper das FCNs os autores propuseram pela primeira vez o uso de **Skip Connections** em redes neurais profundas para reconhecimento de imagens. As Skip Connections nas FCNs alimentam as saídas das primeiras convoluções da rede para as últimas camadas inseridas no lugar das antigas camadas FC. É notável que os outputs dessas primeiras camadas possuem resoluções espaciais muito maiores que a saída da última camada convolucional, permitindo que o processo de inferência possa ser feito diretamente com feature maps de maior resolução espacial, resultando em uma segmentação mais detalhada.

![Skip Connections FCNs](https://www.dropbox.com/s/r9cvc14fju7kvpw/skip_connections.png?dl=1)

Skip Connections em FCNs permitem duas coisas:

1.   Treinamento mais eficiente das primeiras camadas por meio de "atalhos" para o backpropagation, assim como nas ResNets;
2.   Incorporação de informação de alto nível semântico das camadas mais profundas com informação de baixo nível semântico, porém com maior resolução espacial no classificador de pixels da Rede.

Na prática, Skip Connections são implementadas como concatenações no eixo dos canais, como mostrado na figura abaixo. Deve-se notar que a camada posterior deve estar preparada para receber a soma da quantidade de canais provenientes da camada anterior com o número de canais da camada que gerou os dados para o Skip Connection.

![Skip FCN](https://www.dropbox.com/s/af4ao1h7kql4uk6/Nets_FCN_Petro.png?dl=1)

# Atividade Prática

Vamos transformar uma CNN já conhecida por nós, a AlexNet, em uma FCN para segmentação semântica realizando os incrementos indicados em cada um dos passos a seguir. Lembre que segmentação é uma tarefa de classificação de pixels, portanto a loss deve ser a Cross Entropy. A única diferença para a tarefa de classificação da aula passada é que as dimensões espaciais dos labels $(224, 224)$ e das predições da rede $(\#classes, 224, 224)$ devem ser linearizadas para o cálculo da loss, resultado em vetores de dimensões $(224*224)$ e $(\#classes, 224*224)$. Essa linearização deve ser implementada nas funções *train()* e *test()*.

1.   Implemente uma FCN com o backbone de uma [AlexNet](https://pytorch.org/docs/stable/torchvision/models.html#torchvision.models.alexnet), inicialmente sem nenhuma skip connection. Para isso, utilize apenas os blocos convolucionais da AlexNet para extração de features, seguido do bloco classifier onde você implementará apenas blocos convolucionais. É importante adicionar no forward uma operação de [upsample](https://pytorch.org/docs/stable/nn.html#torch.nn.functional.upsample) bilinear, como nas FCN tradicionais, na transição entre o bloco de features e o classifier.

2.   Modifique sua implementação para inserir Skip Connections assim como numa FCN-8s ou uma FCN-16s.

3.   Compare o desempenho de FCNs com e sem Skip Connections e com o backbone estando pré-treinado ou não.

4.   Fazer a mesma adaptação para a VGG 16.

In [ ]:
class FCN_AlexNet(nn.Module):

    def __init__(self, num_classes, pretrained=False, skip=False):

        super(FCN_AlexNet, self).__init__()

        self.skip = skip
        
        if self.skip:
            
            # TO DO: (2.) FCN with Skip Connections (adapted from FCN-8s).
            self.backbone1 = # ...
            self.backbone2 = # ...

            self.classifier = # ...
        else:
            
            # TO DO: (1.) FCN without Skip Connections.
            self.backbone = # ...

            self.classifier = # ...
        
        if not pretrained:
            initialize_weights(self)

    def forward(self, x):

        if self.skip:
            
            # TO DO: (2.) Forward on FCN with Skip Connections.
            
        else:

            # TO DO: (1.) Forward on FCN without Skip Connections.
            
        
        output = # ...

        return output
        
net = FCN_AlexNet(args['n_classes'],
                  pretrained=args['pretrained'],
                  skip=args['skip']).to(args['device'])

print(net)

In [ ]:
# class FCN_VGG16(nn.Module):

#     def __init__(self, num_classes, pretrained=False, skip=False):

#         super(FCN_VGG16, self).__init__()

#         self.skip = skip
        
#         if self.skip:
            
#             # TO DO: (2.) FCN with Skip Connections (adapted from FCN-8s).
#             self.backbone1 = # ...
#             self.backbone2 = # ...

#             self.classifier = # ...
#         else:
            
#             # TO DO: (1.) FCN without Skip Connections.
#             self.backbone = # ...
            
#             self.classifier = # ...
        
#         if not pretrained:
#             initialize_weights(self)

#     def forward(self, x):

#         if self.skip:
            
#             # TO DO: (2.) Forward on FCN with Skip Connections.

#         else:

#             # TO DO: (1.) Forward on FCN without Skip Connections.

#         output = # ...

#         return output
        
# net = FCN_VGG16(args['n_classes'],
#                 pretrained=args['pretrained'],
#                 skip=args['skip']).to(args['device'])

# print(net)

In [ ]:
# Setting optimizer.
if args['pretrained']:
    
    if args['skip']:
        
        # Skip Connections + pretrained.
        # ...

    else:
        
        # No Skip Connections + pretrained.
        optimizer = optim.Adam([{'params': net.backbone.parameters(), 'lr': args['lr'] * 0.1},
                                {'params': net.classifier.parameters()}],
                               lr=args['lr'],
                               weight_decay=args['weight_decay'],
                               betas=(args['momentum'], 0.999))
else:
    
    if args['skip']:

        # Skip Connections + from scratch.
        # ...
        
    else:
        
        # No Skip Connections + from scratch.
        # ...

In [ ]:
# Setting loss.
criterion = #...

In [ ]:
# Training function.
def train(train_loader, net, criterion, optimizer, epoch):

    tic = time.time()
    
    # Setting network for training mode.
    net.train()

    # Lists for losses and metrics.
    train_loss = []
    
    int_all = np.asarray(args['n_classes'], dtype=np.float32)
    uni_all = np.asarray(args['n_classes'], dtype=np.float32)

    # Iterating over batches.
    for i, batch_data in enumerate(train_loader):

        # Obtaining images and labels for batch.
        inps, labs = batch_data

        # Casting to cuda variables.
        inps = inps.to(args['device'])
        labs = labs.to(args['device'])

        # Clears the gradients of optimizer.
        optimizer.zero_grad()

        # Forwarding through network.
        outs = net(inps)

        # TO DO: Computing loss.
        # Remeber to reshape the data
        # ... 

        # TO DO: Computing backpropagation.
        # ...

        # Obtaining predictions.
        prds = outs.data.max(1)[1].squeeze_(1).squeeze(0).cpu().numpy()
        
        # Appending metrics for epoch error calculation.
        int_sum, uni_sum = evaluate([prds],
                                    [labs.detach().squeeze(0).cpu().numpy()])

        int_all = int_all + int_sum
        uni_all = uni_all + uni_sum

        # Updating loss meter.
        train_loss.append(loss.data.item())

    toc = time.time()
    
    # Transforming list into numpy array.
    train_loss = np.asarray(train_loss)
    
    # Computing error metrics for whole epoch.
    iou = 0
    iou = np.divide(int_all, uni_all)
#     print(iou)

    # Printing training epoch loss and metrics.
    print('-------------------------------------------------------------------')
    print('[epoch %d], [train loss %.4f +/- %.4f], [miou %.4f], [time %.4f]' % (
        epoch, train_loss.mean(), train_loss.std(), iou[1:].mean(), (toc - tic)))
    print('-------------------------------------------------------------------')

def validate(val_loader, net, criterion, epoch):

    tic = time.time()
    
    # Setting network for evaluation mode.
    net.eval()

    # Lists for losses and metrics.
    val_loss = []

    int_all = np.asarray(args['n_classes'], dtype=np.float32)
    uni_all = np.asarray(args['n_classes'], dtype=np.float32)

    # Iterating over batches.
    for i, batch_data in enumerate(val_loader):

        # Obtaining images and labels for batch.
        inps, labs = batch_data

        # Casting to cuda variables.
        inps = inps.to(args['device'])
        labs = labs.to(args['device'])

        # Forwarding through network.
        outs = net(inps)

        # TO DO: Computing loss.
        # ...

        # Obtaining predictions.
        prds = outs.data.max(1)[1].squeeze_(1).squeeze(0).cpu().numpy()

        # Appending metrics for epoch error calculation.
        int_sum, uni_sum = evaluate([prds],
                                    [labs.detach().squeeze(0).cpu().numpy()])

        int_all = int_all + int_sum
        uni_all = uni_all + uni_sum

        # Updating loss meter.
        val_loss.append(loss.data.item())
        
        if i == 0 and epoch % args['show_freq'] == 0:
            
            fig, ax = plt.subplots(1, 3, figsize=(15, 5))
            
            ax[0].imshow(inps[0].detach().cpu().numpy().transpose(1, 2, 0))
            ax[0].set_yticks([])
            ax[0].set_xticks([])
            ax[0].set_title('Image')
            
            ax[1].imshow(prds * 127,
                         cmap=plt.get_cmap('gray'),
                         vmin=0, vmax=255)
            ax[1].set_yticks([])
            ax[1].set_xticks([])
            ax[1].set_title('Prediction')
            
            ax[2].imshow(labs[0].detach().squeeze(0).cpu().numpy() * 127,
                         cmap=plt.get_cmap('gray'),
                         vmin=0, vmax=255)
            ax[2].set_yticks([])
            ax[2].set_xticks([])
            ax[2].set_title('True Label')
            
            plt.show()

    toc = time.time()
    
    # Transforming list into numpy array.
    val_loss = np.asarray(val_loss)
    
    # Computing error metrics for whole epoch.
    iou = 0
    iou = np.divide(int_all, uni_all)
#     print(iou)

    # Printing test epoch loss and metrics.
    print('-------------------------------------------------------------------')
    print('[epoch %d], [test loss %.4f +/- %.4f], [miou %.4f], [time %.4f]' % (
        epoch, val_loss.mean(), val_loss.std(), iou[1:].mean(), (toc - tic)))
    print('-------------------------------------------------------------------')

In [ ]:
# Iterating over epochs.
for epoch in range(1, args['epoch_num'] + 1):

    # Training function.
    train(train_loader, net, criterion, optimizer, epoch)

    # Computing test loss and metrics.
    validate(val_loader, net, criterion, epoch)